In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import talib

In [30]:
# Define a function to fetch stock data from yfinance
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [31]:
# Define a function to preprocess the data and extract the four stock indicators
def preprocess_data(stock_data):
    stock_data['RSI'] = talib.RSI(stock_data['Close'])
    stock_data['EMA50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
    stock_data['EMA200'] = stock_data['Close'].ewm(span=200, adjust=False).mean()
    stock_data['EMA13'] = stock_data['Close'].ewm(span=13, adjust=False).mean()
    stock_data['EMA26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
    stock_data['MACD'] = talib.MACD(stock_data['Close'])[0]
    stock_data = stock_data.dropna()
    return stock_data[['RSI', 'EMA50', 'EMA200', 'EMA13', 'EMA26', 'MACD']]

In [32]:
# Define a function to build and train the model
def build_and_train_model(X_train, y_train):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model

In [33]:
# Fetch AAPL stock data from year 2000
start_date = '2000-01-01'
end_date = '2023-03-27'
aapl_data = fetch_stock_data('AAPL', start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [34]:
# Preprocess AAPL stock data and extract the four stock indicators
preprocessed_aapl_data = preprocess_data(aapl_data)

In [35]:
# Train the model on multiple other stocks
stocks = ['MSFT', 'AMZN','GOOG', 'TSLA']
start_date = '2010-01-01'
end_date = '2023-03-27'

In [36]:
# Create empty dataframes to store X and y data
X = pd.DataFrame()
y = pd.Series()

/tmp/ipykernel_7427/3737762291.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y = pd.Series()


In [37]:
# Loop through the list of stocks and preprocess the data
for stock in stocks:
    stock_data = yf.download(stock, start=start_date, end=end_date)
    preprocessed_data = preprocess_data(stock_data)
    X = pd.concat([X, preprocessed_data])
    y = pd.concat([y, pd.Series(np.where(preprocessed_data['RSI'].shift(-1) >= 60, -1, 1), index=preprocessed_data.index[:].dropna())])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
# Build and train the model
model = build_and_train_model(X_train, y_train)

In [40]:

# Use the model to make predictions on AAPL stock data
aapl_data = aapl_data.dropna()
predictions = model.predict(preprocessed_aapl_data)


In [42]:
aapl_data['Prediction'] = predictions
aapl_data['Signal'] = np.where(aapl_data['Prediction'].rolling(window=3).sum() >= 2, 1, np.where(aapl_data['Prediction'].rolling(window=3).sum() <= -2, -1, 0))
aapl_data['BuySell'] = np.where(aapl_data['Signal'] == 1, 'BUY', np.where(aapl_data['Signal'] == -1, 'SELL', ''))

In [43]:
initial_investment = 100000
aapl_data['Shares'] = np.floor(initial_investment / aapl_data['Open'])
aapl_data['Profit'] = aapl_data['Shares'] * aapl_data['Close'] - initial_investment
total_profit = aapl_data['Profit'].sum()

print('Total profit:', round(total_profit, 2))

Total profit: 150828.2
